In [2]:
%%writefile requirements.txt
llama-index
llama-index-llms-huggingface
llama-index-embeddings-huggingface
chromadb
llama-index-vector-stores-chroma
llama-index-llms-groq
einops
accelerate
sentence-transformers
llama-index-llms-mistralai
llama-index-embeddings-mistralai
chainlit


Writing requirements.txt


In [3]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 559.5/559.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install --upgrade huggingface_hub

In [25]:
!pip install chainlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 8.3 MB/s eta 0:00:00
  Created wheel for literalai: filename=literalai-0.0.607-py3-none-any.whl size=64143 sha256=dacb0ea30083e01bb729acfc56c5f1233bc2e315cbae900567072560bdf25e01
  Stored in directory: /root/.cache/pip/w

In [5]:
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex,SummaryIndex
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.tools import FunctionTool,QueryEngineTool
from llama_index.core.vector_stores import MetadataFilters,FilterCondition
from typing import List,Optional

#Define embedding model and LLM


In [19]:
import json
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
#from llama_index.core import Settings
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding

config_data = json.load(open("config.json"))
HF_TOKEN = config_data["HF_TOKEN"]
embed_model = MistralAIEmbedding(model_name="mistral-embed", api_key="H1rVHKqTjonaHi4vOZ8jozgwtrhRiSjZ")

llm = MistralAI(model = 'mistral-large-latest', api_key = "H1rVHKqTjonaHi4vOZ8jozgwtrhRiSjZ")

#Read file

In [46]:
documents = SimpleDirectoryReader(input_files=['Harry Potter The Complete Collection.pdf']).load_data(num_workers=4)


/usr/local/lib/python3.10/dist-packages/llama_index/core/readers/file/base.py:663: UserWarning: Specified num_workers exceed number of CPUs in the system. Setting `num_workers` down to the maximum CPU count.
  warnings.warn(


#Chunk the document

In [47]:
splitter = SentenceSplitter(chunk_size=1024,chunk_overlap=100)
nodes = splitter.get_nodes_from_documents(documents)
print(f"Length of nodes : {len(nodes)}")
print(f"get the content for node 0 :{nodes[0].get_content(metadata_mode='all')}")

Length of nodes : 3623
get the content for node 0 :page_label: 1
file_name: Harry Potter The Complete Collection.pdf
file_path: Harry Potter The Complete Collection.pdf
file_type: application/pdf
file_size: 17338552
creation_date: 2024-07-07
last_modified_date: 2024-07-07


#Define relevant contexts


In [48]:
from llama_index.core import ServiceContext, StorageContext

service_context = ServiceContext.from_defaults(llm = llm,
                                               embed_model = embed_model)
                                               #callback_manager = CallbackManager([cl.LlamaIndexCallbackHandler()])


<ipython-input-48-fb749a54b22e>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm = llm,


#Create vector store index


In [49]:
vector_index = VectorStoreIndex.from_documents(documents,
                                               #storage_context=storage_context
                                               service_context=service_context,
                                               node_parser=nodes,
                                               show_progress=True)

vector_index.storage_context.persist(persist_dir="./storage")

Parsing nodes:   0%|          | 0/3623 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/2048 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/1575 [00:00<?, ?it/s]

#Create the query engine (for running on CLI)

In [50]:
query_engine = vector_index.as_query_engine(similarity_top_k = 3,
                                            service_context=service_context)

response = query_engine.query("Who is the main character?")
print(str(response))

The main character is a boy named Harry Potter, although he is not directly mentioned in the provided context. The context refers to the Dursleys, who have a secret related to the Potters, and it is mentioned that the Potters have a small son. The file path "Harry Potter The Complete Collection.pdf" also suggests that the main character is Harry Potter.


#Deploy the engine on Chainlit

In [51]:
%%writefile app.py
from llama_index.core.callbacks.base import CallbackManager
from llama_index.core import load_index_from_storage, StorageContext, ServiceContext
from llama_index.llms.mistralai import MistralAI
from llama_index.embeddings.mistralai import MistralAIEmbedding
import chainlit as cl

@cl.on_chat_start
async def start_chat():
  embed_model = MistralAIEmbedding(model_name="mistral-embed", api_key="H1rVHKqTjonaHi4vOZ8jozgwtrhRiSjZ")

  llm = MistralAI(model = 'mistral-large-latest', api_key = "H1rVHKqTjonaHi4vOZ8jozgwtrhRiSjZ")
  storage_context = StorageContext.from_defaults(persist_dir="./storage")

  service_context = ServiceContext.from_defaults(llm = llm,
                                                 embed_model = embed_model,
                                                 callback_manager = CallbackManager([cl.LlamaIndexCallbackHandler()]))

  index = load_index_from_storage(
      service_context = service_context,
      storage_context = storage_context,
  )

  query_engine = index.as_query_engine(similarity_top_k = 3,
                                       service_context = service_context)

  cl.user_session.set("chatbot", query_engine)

  await cl.Message(
      author="Assistant", content="Hello! What do you want to know about Harry Potter?"
  ).send()

@cl.on_message
async def main(message: cl.Message):
  query_engine = cl.user_session.get("chatbot")

  response = await cl.make_async(query_engine.query)(message.content)

  response_message = cl.Message(content="", author="Assistant")

  for token in response.response:
    await response_message.stream_token(token=token)

  await response_message.send()

Overwriting app.py


In [52]:
!fuser -n tcp -k 8000


from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(8000)"))

https://z9ohyrwfn0d-496ff2e9c6d22116-8000-colab.googleusercontent.com/


In [53]:
!chainlit run app.py

2024-07-07 18:19:26 - Your app is available at http://localhost:8000
2024-07-07 18:19:32 - Translated markdown file for en-US not found. Defaulting to chainlit.md.
2024-07-07 18:20:21 - Loading all indices.
2024-07-07 18:20:21 - HTTP Request: POST https://api.mistral.ai/v1/embeddings "HTTP/1.1 200 OK"
2024-07-07 18:20:21 - Task exception was never retrieved
future: <Task finished name='Task-60' coro=<Step.update() done, defined at /usr/local/lib/python3.10/dist-packages/chainlit/step.py:275> exception=TypeError("unsupported operand type(s) for /: 'PosixPath' and 'NoneType'")>
Traceback (most recent call last):
  File "/usr/lib/python3.10/asyncio/tasks.py", line 234, in __step
    result = coro.throw(exc)
  File "/usr/local/lib/python3.10/dist-packages/chainlit/step.py", line 296, in update
    await asyncio.gather(*tasks)
  File "/usr/lib/python3.10/asyncio/tasks.py", line 304, in __wakeup
    future.result()
  File "/usr/lib/python3.10/asyncio/tasks.py", line 232, in __step
    result